In [314]:
import pandas as pd
import seaborn as sns
import random
import numpy as np
import math
iris = pd.read_csv('iris.csv')

class ProcessingData:
    
    @staticmethod
    def shuffle(base):
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalize(base):
        base.iloc[:,:-1] = (base.iloc[:,:-1]-base.iloc[:,:-1].min())/(base.iloc[:,:-1].max()-base.iloc[:,:-1].min())

    @staticmethod
    def devide(base):
        n = int(len(base)*0.7)
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

class Bayes:
    
    @staticmethod
    def recoverTreningData(data):
        sortedData = []
        sort = data.groupby(data.variety)
        sortedData.append(sort.get_group("Setosa"))
        sortedData.append(sort.get_group("Versicolor"))
        sortedData.append(sort.get_group("Virginica"))

        return sortedData

    @staticmethod
    def meanMaker(sortedData):
        meanTable = []
        for j in range(len(sortedData)):
            meanTable.append(sortedData[j].mean(numeric_only=True).tolist())
        return meanTable

    @staticmethod
    def make10Table(data,meanTable):
        oneZeroTable = []
        for j in range(len(data)):
            speciesTable = []
            for k in range(len(data[j].columns)-1):
                smaller = data[j][data[j].iloc[:,k] < meanTable[j][k]].count()
                bigger = len(data[j]) - smaller
                
                if smaller[k] > bigger[k]:
                    speciesTable.extend([1,0])
                else:
                     speciesTable.extend([0,1])
            oneZeroTable.append(speciesTable)
        return oneZeroTable

    @staticmethod
    def softSet(validate_data,oneZeroTable,meanTable):
        correct = 0
        varieties = ["Setosa","Versicolor","Virginica"]
        for i in range(len(validate_data)):
            trueName = validate_data.iloc[i].variety
            oneZeroValidate = []
            for j in range(len(meanTable)):
                oneZeroValidateOne = []
                for k in range(len(validate_data.iloc[i])-1):
                    if validate_data.iloc[i][k] < meanTable[j][k]:
                        oneZeroValidateOne.extend([1,0])
                    else:
                        oneZeroValidateOne.extend([0,1])
                oneZeroValidate.append(oneZeroValidateOne)
                 
            tableOfSums = []
            for j in range(len(oneZeroValidate)):
                suma = 0
                for sum in range(len(oneZeroValidate[j])):
                    suma+=oneZeroValidate[j][sum]*oneZeroTable[j][sum]
                tableOfSums.append(suma)
                

            guessedName = varieties[np.argmax(tableOfSums)]

            if guessedName == trueName:
                correct+=1
                
        return correct,correct/len(validate_data)*100


In [315]:
irisEdit = iris.copy()

ProcessingData.shuffle(irisEdit)
ProcessingData.normalize(irisEdit)
trening_data,validate_data = ProcessingData.devide(irisEdit)

In [316]:
sortedData = Bayes.recoverTreningData(trening_data)

In [317]:
meanTable = Bayes.meanMaker(sortedData)

In [318]:
oneZeroTable = Bayes.make10Table(sortedData,meanTable)

In [319]:
correct, accuracy = Bayes.softSet(validate_data,oneZeroTable,meanTable)
print("{}/{} accuracy => {:.2f}%".format(correct,len(validate_data),accuracy))

21/45 accuracy => 46.67%
